# Abstract Interface (V3)
### Machine precision tests for solidity vetting

In [1]:
from uniswappy import *

In [2]:
fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [3]:
user_nm = 'user_machine_check'
eth_amount = UniV3Helper().dec2gwei(1000)
dai_amount = UniV3Helper().dec2gwei(100000) 

### 1. Check abstract interface

In [4]:
eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   precision = UniswapExchangeData.TYPE_GWEI,
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

out_v3 = Join().apply(lp, user_nm, eth_amount, dai_amount, lwr_tick, upr_tick)
lp.summary()

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999999999999999999457, TKN = 99999999999999999999457
Gross Liquidity: 10000000000000000000000 



In [5]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011", precision = UniswapExchangeData.TYPE_GWEI)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

AddLiquidity().apply(lp, eth, user_nm, UniV3Helper().dec2gwei(10))
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000000000000000000000, DAI = 100000000000000000000000
Liquidity: 10000000000000000000000 

Exchange ETH-DAI (LP)
Reserves: ETH = 1010000000000000000000, DAI = 101000000000000000000000
Liquidity: 10100000000000000000000 



In [6]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011", precision = UniswapExchangeData.TYPE_GWEI)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

RemoveLiquidity().apply(lp, eth, user_nm, UniV3Helper().dec2gwei(100))
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000000000000000000000, DAI = 100000000000000000000000
Liquidity: 10000000000000000000000 

Exchange ETH-DAI (LP)
Reserves: ETH = 900000000000000000000, DAI = 90000000000000000000000
Liquidity: 9000000000000000000000 



In [7]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011", precision = UniswapExchangeData.TYPE_GWEI)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

Swap().apply(lp, dai, user_nm, UniV3Helper().dec2gwei(1000))
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000000000000000000000, DAI = 100000000000000000000000
Liquidity: 10000000000000000000000 

Exchange ETH-DAI (LP)
Reserves: ETH = 990128419656029387011, DAI = 101000000000000000000000
Liquidity: 10000000000000000000000 



In [8]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011", precision = UniswapExchangeData.TYPE_GWEI)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

WithdrawSwap().apply(lp, eth, user_nm, UniV3Helper().dec2gwei(10))
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000000000000000000000, DAI = 100000000000000000000000
Liquidity: 10000000000000000000000 

Exchange ETH-DAI (LP)
Reserves: ETH = 989999999999999999241, DAI = 100000000000000000000000
Liquidity: 9949799447405356572672 



### 2. Check quotes

In [9]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011", precision = UniswapExchangeData.TYPE_GWEI)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000000000000000000000, DAI = 100000000000000000000000
Liquidity: 10000000000000000000000 



In [10]:
p_eth = LPQuote().get_price(lp, eth)
p_dai = LPQuote().get_price(lp, dai)
print(f'The price of {eth.token_name} in {dai.token_name} is {p_eth}')
print(f'The price of {dai.token_name} in {eth.token_name} is {p_dai}')

The price of ETH in DAI is 100.0
The price of DAI in ETH is 0.01


In [11]:
amt_eth = LPQuote(include_fee = True).get_amount(lp, eth, UniV3Helper().dec2gwei(1))
amt_dai = LPQuote(include_fee = True).get_amount(lp, dai, UniV3Helper().dec2gwei(1))
print(f'{UniV3Helper().dec2gwei(1)} {eth.token_name} token is worth {amt_dai} {dai.token_name} after swap fees')
print(f'{UniV3Helper().dec2gwei(1)} {dai.token_name} token is worth {amt_eth} {eth.token_name} after swap fees')

1000000000000000000 ETH token is worth 9969900600091018 DAI after swap fees
1000000000000000000 DAI token is worth 99600698103990321650 ETH after swap fees


In [12]:
amt_eth = LPQuote(False).get_amount_from_lp(lp, eth, UniV3Helper().dec2gwei(1))
amt_dai = LPQuote().get_amount_from_lp(lp, eth, UniV3Helper().dec2gwei(1))
print(f'{UniV3Helper().dec2gwei(1)} LP token is worth {amt_eth} {eth.token_name}')
print(f'{UniV3Helper().dec2gwei(1)} LP token is worth {amt_dai} {dai.token_name}')

1000000000000000000 LP token is worth 199690059907017973 ETH
1000000000000000000 LP token is worth 1.9969005990701797e+19 DAI
